In [6]:
import pandas as pd
import zipfile
import json
import numpy as np

In [7]:
import json


import requests

#calling api 
response = requests.get("https://gbfs.urbansharing.com/oslobysykkel.no/station_information.json").json()

def formating(json):
    """function transforming data in flat json format
    """
    transformed_data = []

    for element in json['data']['stations']:
        transformed_data.append(
        {"station_id": int(element['station_id']), 
        "name": element['name'],
        "address": element['address'],
        "android": element['rental_uris']['android'],
        "ios": element['rental_uris']['ios'],
        "lat": element['lat'],
        "lon": element['lon'],
        "capacity": element['capacity']
        })
    return transformed_data

def station_api():
    """Returns a set data structure with all the current stations
    from the api call
    """
    data_transformed = formating(response)
    station_check_list = [int(row['station_id']) for row in data_transformed]
    return station_check_list

stations = formating(response)

In [8]:
print(stations)

[{'station_id': 2355, 'name': 'Økern T-bane', 'address': 'Økernveien 147', 'android': 'oslobysykkel://stations/2355', 'ios': 'oslobysykkel://stations/2355', 'lat': 59.928894918817605, 'lon': 10.806234776281599, 'capacity': 12}, {'station_id': 2350, 'name': 'Blindern T-Bane', 'address': 'Apalveien 60', 'android': 'oslobysykkel://stations/2350', 'ios': 'oslobysykkel://stations/2350', 'lat': 59.94022899411701, 'lon': 10.716856460117071, 'capacity': 25}, {'station_id': 2349, 'name': 'Maritimt Museum', 'address': 'Bygdøynesveien 37', 'android': 'oslobysykkel://stations/2349', 'ios': 'oslobysykkel://stations/2349', 'lat': 59.902942924651484, 'lon': 10.698048967006343, 'capacity': 21}, {'station_id': 2347, 'name': 'Huk 2', 'address': 'Strømsborgveien 48', 'android': 'oslobysykkel://stations/2347', 'ios': 'oslobysykkel://stations/2347', 'lat': 59.89843428245928, 'lon': 10.674398461025874, 'capacity': 30}, {'station_id': 2340, 'name': 'Tiedemannsparken', 'address': 'Hovinveien 43', 'android': '

In [9]:
#access the json files with travel data
all_files_data = []

with zipfile.ZipFile(r'turhistorikk.zip', 'r') as zip_ref:
    for file_name in zip_ref.namelist():
        # print(file_name)
        with zip_ref.open(file_name, 'r') as file:
            data = json.load(file)
            # print(len(data))

            for l in data:

                start_station = int(l['start_station_id'])
                l['start_station_id'] = start_station
                
                end_station = int(l['end_station_id'])
                l['end_station_id'] = end_station
                all_files_data.append(l)
                

print(len(all_files_data))

1358236


In [10]:
station = pd.DataFrame(stations)

print(station.columns)

Index(['station_id', 'name', 'address', 'android', 'ios', 'lat', 'lon',
       'capacity'],
      dtype='object')


In [11]:

turhistorikk = pd.DataFrame(all_files_data)

print(turhistorikk.columns)


Index(['started_at', 'ended_at', 'duration', 'start_station_id',
       'start_station_name', 'start_station_description',
       'start_station_latitude', 'start_station_longitude', 'end_station_id',
       'end_station_name', 'end_station_description', 'end_station_latitude',
       'end_station_longitude'],
      dtype='object')


### Predicting availability the next hour for each station
We are focusing on predicting availability the next hour 
by focusing on each stations and looking at how many trips started each hour.
This will be a easy and cost efficient approach 

there are possibilities to include end station data as parameter that will contribute to the accuracy 
but it is important to consider the trade-offs between model accuracy and model complexity when deciding which features to include in the analysis.

In [12]:
#removing unnecessary columns
turhistorikk = turhistorikk[['started_at', 'start_station_id',  'start_station_name', 
                            'start_station_latitude', 'start_station_longitude',
                            'duration']]

#removing micro seconds in column started_at in order to transform to datetime object

ls = []
for row in turhistorikk['started_at']:
    ls.append(row[:19])
    
turhistorikk['started_at'] = ls 


In [13]:
#transform start_at column to datime object
turhistorikk['started_at'] = pd.to_datetime(turhistorikk['started_at'], format="%Y-%m-%d %H:%M:%S")

turhistorikk_cop = turhistorikk.copy()

In [14]:
# Group by start_station_id and timestamp to get the count of trips
trip_counts = turhistorikk_cop.groupby(['start_station_id',
                                    pd.Grouper(key='started_at', freq='H')])['start_station_id'].count().reset_index(name='trip_count')

#['started_at','start_station_id','start_station_name', 'start_station_latitude', 'start_station_longitude', 'duration']

In [15]:
trip_counts['date'] = trip_counts['started_at'].dt.date
trip_counts['hour'] = trip_counts['started_at'].dt.hour
trip_counts

,start_station_id,started_at,trip_count,date,hour
0,377,2022-01-02 12:00:00,1,2022-01-02,12
1,377,2022-01-02 13:00:00,1,2022-01-02,13
2,377,2022-01-04 16:00:00,1,2022-01-04,16
3,377,2022-01-06 06:00:00,1,2022-01-06,6
4,377,2022-01-06 10:00:00,1,2022-01-06,10
...,...,...,...,...,...
652892,2355,2023-03-28 14:00:00,1,2023-03-28,14
652893,2355,2023-03-29 12:00:00,1,2023-03-29,12
652894,2355,2023-03-29 18:00:00,1,2023-03-29,18
652895,2355,2023-03-30 08:00:00,1,2023-03-30,8


In [46]:
# test = turhistorikk.groupby(['start_station_id'])['start_station_id'].count()
trip_counts_cop = trip_counts.copy()
trip_counts_cop['trip_count']

typing.Literal[<no_default>]
0         1
1         1
2         1
3         1
4         1
         ..
652892    1
652893    1
652894    1
652895    1
652896    2
Name: trip_count, Length: 652897, dtype: int64

In [47]:
df_pivot = trip_counts_cop.pivot_table(values='trip_count', index=['start_station_id', 'date'], columns='hour', aggfunc='sum', fill_value=0)


In [48]:
df_pivot.head(10)

hour                         0   1   2   3   4   5   6   7   8   9   ...  14   
start_station_id date                                                ...       
377              2022-01-02   0   0   0   0   0   0   0   0   0   0  ...   0  \
                 2022-01-04   0   0   0   0   0   0   0   0   0   0  ...   0   
                 2022-01-06   0   0   0   0   0   0   1   0   0   0  ...   0   
                 2022-01-07   0   0   0   0   0   0   1   1   0   0  ...   1   
                 2022-01-08   0   0   0   0   0   0   0   0   0   0  ...   0   
                 2022-01-09   0   0   0   0   0   0   0   0   0   0  ...   1   
                 2022-01-10   0   0   0   0   0   1   0   0   0   0  ...   0   
                 2022-01-12   0   0   0   0   0   0   0   0   0   0  ...   1   
                 2022-01-13   0   0   0   0   0   0   0   1   0   0  ...   0   
                 2022-01-15   0   0   0   0   0   0   0   0   0   0  ...   0   

hour                         15  16  17  18  19  20  21  22  23  
start_station_id date                                            
377              2022-01-02   0   0   0   0   0   0   0   0   0  
                 2022-01-04   0   1   0   0   0   0   0   0   0  
                 2022-01-06   0   0   0   0   0   0   0   0   0  
                 2022-01-07   0   0   0   0   0   0   0   0   0  
                 2022-01-08   0   0   0   1   0   0   0   0   0  
                 2022-01-09   0   0   0   0   0   0   0   0   0  
                 2022-01-10   0   0   0   0   0   0   0   0   0  
                 2022-01-12   0   0   0   0   0   0   0   0   0  
                 2022-01-13   0   0   0   0   0   0   0   0   0  
                 2022-01-15   0   0   1   0   0   0   0   0   0  

[10 rows x 24 columns]

In [49]:
df_pivot.columns = [f'hour_{hour}' for hour in df_pivot]
df_pivot

hour_0  hour_1  hour_2  hour_3  hour_4  hour_5   
start_station_id date                                                         
377              2022-01-02       0       0       0       0       0       0  \
                 2022-01-04       0       0       0       0       0       0   
                 2022-01-06       0       0       0       0       0       0   
                 2022-01-07       0       0       0       0       0       0   
                 2022-01-08       0       0       0       0       0       0   
...                             ...     ...     ...     ...     ...     ...   
2355             2023-03-27       0       0       0       0       0       0   
                 2023-03-28       0       0       0       0       0       0   
                 2023-03-29       0       0       0       0       0       0   
                 2023-03-30       0       0       0       0       0       0   
                 2023-03-31       0       0       0       0       0       0   

                             hour_6  hour_7  hour_8  hour_9  ...  hour_14   
start_station_id date                                        ...            
377              2022-01-02       0       0       0       0  ...        0  \
                 2022-01-04       0       0       0       0  ...        0   
                 2022-01-06       1       0       0       0  ...        0   
                 2022-01-07       1       1       0       0  ...        1   
                 2022-01-08       0       0       0       0  ...        0   
...                             ...     ...     ...     ...  ...      ...   
2355             2023-03-27       0       0       0       0  ...        0   
                 2023-03-28       0       0       0       0  ...        1   
                 2023-03-29       0       0       0       0  ...        0   
                 2023-03-30       0       0       1       0  ...        0   
                 2023-03-31       0       0       0       0  ...        0   

                             hour_15  hour_16  hour_17  hour_18  hour_19   
start_station_id date                                                      
377              2022-01-02        0        0        0        0        0  \
                 2022-01-04        0        1        0        0        0   
                 2022-01-06        0        0        0        0        0   
                 2022-01-07        0        0        0        0        0   
                 2022-01-08        0        0        0        1        0   
...                              ...      ...      ...      ...      ...   
2355             2023-03-27        0        0        0        0        0   
                 2023-03-28        0        0        0        0        0   
                 2023-03-29        0        0        0        1        0   
                 2023-03-30        0        0        0        0        0   
                 2023-03-31        0        0        0        0        0   

                             hour_20  hour_21  hour_22  hour_23  
start_station_id date                                            
377              2022-01-02        0        0        0        0  
                 2022-01-04        0        0        0        0  
                 2022-01-06        0        0        0        0  
                 2022-01-07        0        0        0        0  
                 2022-01-08        0        0        0        0  
...                              ...      ...      ...      ...  
2355             2023-03-27        0        0        0        0  
                 2023-03-28        0        0        0        0  
                 2023-03-29        0        0        0        0  
                 2023-03-30        0        0        0        0  
                 2023-03-31        0        0        0        0  

[99329 rows x 24 columns]

In [50]:
df_pivot = df_pivot.reset_index()
df_pivot

,start_station_id,date,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,377,2022-01-02,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,377,2022-01-04,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,377,2022-01-06,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,377,2022-01-07,0,0,0,0,0,0,1,1,...,1,0,0,0,0,0,0,0,0,0
4,377,2022-01-08,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99324,2355,2023-03-27,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99325,2355,2023-03-28,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
99326,2355,2023-03-29,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
99327,2355,2023-03-30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
df_pivot.columns

Index(['start_station_id', 'date', 'hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23'],
      dtype='object')